In [22]:
import numpy as np
from scipy import optimize
import os
import subprocess
from contextlib import contextmanager
import time


First, we'll read in the files created by the Fortran HF code. Things are a little messy thanks to the way I stored things in the past, but they are standard numpy arrays after this cell. Take note of the ordering and suggest a more agreeable choice if you've got an opinion.

In [3]:
def read_wfs(restartfile):
    #restartfile = '../HF/wf_numpy.bin'

    intchunk = np.dtype(np.int32).itemsize
    realchunk = np.dtype(np.float64).itemsize
    h = np.fromfile(restartfile, dtype=np.float64, count=1, offset=0)[0]
    nbox = np.fromfile(restartfile, dtype=np.int32, count=1, offset=(realchunk+0*intchunk))[0]
    lmax = np.fromfile(restartfile, dtype=np.int32, count=1, offset=(realchunk+1*intchunk))[0]
    nmax = np.fromfile(restartfile, dtype=np.int32, count=1, offset=(realchunk+2*intchunk))[0]
#     print("hbox = ", h)
#     print("nbox = ",nbox)
#     print("lmax = ",lmax)
#     print("nmax = ",nmax)

    #sortstates, sortenergies, wfr,
    sortstates = np.fromfile(restartfile, dtype=np.int32, count=nmax*3*2, offset=(realchunk+3*intchunk)).reshape(2,3,nmax)
    sortenergies = np.fromfile(restartfile, dtype=np.float64, count=nmax*2, offset=(realchunk+3*intchunk+nmax*3*2*intchunk)).reshape(2,nmax)
    wfs = np.fromfile(restartfile, dtype=np.float64, count=2*2*(lmax+1)*lmax*(nbox+1), offset=(realchunk+3*intchunk+nmax*3*2*intchunk+sortenergies.nbytes)).reshape(2,2,(lmax+1),lmax,(nbox+1))

    return h,nbox,lmax,nmax,sortstates,sortenergies,wfs

def print_wfs(restartfile,h,nbox,lmax,nmax,sortstates,sortenergies,wfs):
    with open(restartfile, 'wb') as file:
        file.write(h)
        file.write(nbox)
        file.write(lmax)
        file.write(nmax)
        file.write(sortstates.reshape((nmax*3*2,)))
        file.write(sortenergies.reshape((nmax*2,)))
        file.write(wfs.reshape((2*2*(lmax+1)*lmax*(nbox+1),)))

    return

def read_hpsi(restartfile):

    intchunk = np.dtype(np.int32).itemsize

    nbox = np.fromfile(restartfile, dtype=np.int32, count=1, offset=(0*intchunk))[0]
    lmax = np.fromfile(restartfile, dtype=np.int32, count=1, offset=(1*intchunk))[0]
    nmax = np.fromfile(restartfile, dtype=np.int32, count=1, offset=(2*intchunk))[0]

    sortenergies = np.fromfile(restartfile, dtype=np.float64, count=nmax*2, offset=(3*intchunk)).reshape(2,nmax)
    hpsi = np.fromfile(restartfile, dtype=np.float64, count=2*2*(lmax+1)*lmax*(nbox+1), offset=(3*intchunk+sortenergies.nbytes)).reshape(2,2,(lmax+1),lmax,(nbox+1))

    return sortenergies,hpsi

def get_hpsi(params,h,nbox,lmax,nmax,sortstates,sortenergies,wf):
    restartfile = '../HF/wf_numpy.bin'
    print_wfs(restartfile,h,nbox,lmax,nmax,sortstates,sortenergies,wf)
    make_input(params,1)
    #with cwd('../HF/'):
    oldpwd=os.getcwd()
    os.chdir('../HF/')
    errors.append(run_HF("None",1))
    os.chdir(oldpwd)
    restartfile = '../HF/hpsi.bin'
    sortenergies,hpsi = read_hpsi(restartfile)

    return hpsi

In [4]:
@contextmanager
def cwd(path):
    oldpwd=os.getcwd()
    os.chdir(path)
    try:
        yield
    finally:
        os.chdir(oldpwd)

def spin(spin):
    if(spin==1): return -0.5
    return 0.5

def make_input(params,restart):
    in_template = "../HF/in.template"
    f1 = open(in_template, 'r')
    
    int_string = "&interaction t0= "+str(params[0])+",x0=0.77600,t1=484.2300,x1=-0.31700,t2=-556.6900,x2=-1.0,t3=13757.0,x3=1.2630,w0=125.0,sig=6.0,icoul=1,icm=1,j2terms=.true. / \n"
    if(restart):
        io_string = "&io printwf=F,printdens=F,restartwf=T,printhpsi=T /"
    else:
        io_string = "&io printwf=T,printdens=F,restartwf=F,printhpsi=F /"
    in_out = "../HF/in"
    f2 = open(in_out, 'w')
    f1.seek(0)
    f2.write(f1.read())
    f2.write(int_string)
    f2.write(io_string)
    f2.write("\n")

    f1.close()
    f2.close()

    return

def run_HF(tag,restart):
    noNaN = 1
    if(restart == 0):
        os.system("cp in results/in."+tag)
        #print(os.system("../HF/./main"))
        output = subprocess.run(["./main"], capture_output=True)
        #print(output)
        #print(subprocess.run("../HF/./main",shell=True))
        os.system("mv out results/out."+tag)
        os.system("mv wf_numpy.bin results/wf_numpy.bin."+tag)
        noNaN = os.system("grep -q NaN results/out."+tag)
    else:
        output = subprocess.run(["./main"], capture_output=True)
        #os.system("mv out results/out_rst."+tag)
        #os.system("mv hpsi.bin results/hpsi.bin."+tag)
        #print(output)
        #noNaN = os.system("grep -q NaN results/out_rst."+tag)
    if (noNaN == 0):
        print("NaNs in run {tag}!".format(tag=tag))
        return 1

    return 0


The following cell defines a list of parameters (only one right now) and then loops over those parameters and runs the HF code to generate outputs and WFs.

The inputs, outputs, and WFs are copied to a results folder (this can be changed) with some tag appended to the file name. In the future we may want to change how this tag is generated.

In [7]:
params = [[-2483.45],[-2485.45],[-2453.45],[-2583.45],[-1983.45],]

errors = []

for i in range(len(params)):
    make_input(params[i],0)
    with cwd('../HF/'):
        errors.append(run_HF(str(i),0))




NaNs in run 4!


Now we read the files back in and print the energies of the single particle states to see whats up.

In [8]:
params = [[-2483.45],[-2485.45],[-2453.45],[-2583.45],[-1983.45],]

train_wfs = []
states = []
energies = []
levels = [0,0]
for i in range(len(params)):
    if(errors[i] == 1): continue
    restartfile = '../HF/results/wf_numpy.bin.'+str(i)
    h,nbox,lmax,nmax,sortstates,sortenergies,wf = read_wfs(restartfile)
    train_wfs.append(wf)
    states.append(sortstates)
    hpsi = get_hpsi(params[i],h,nbox,lmax,nmax,sortstates,sortenergies,wf)

    #Calculate energies as a test for levels
    energy = [[],[]]
    for isospin in range(2):
        occupied = 0
        for i in range(lmax):
            level = i
            i_s = sortstates[isospin,2,level]-1
            i_l = sortstates[isospin,1,level]
            i_n = sortstates[isospin,0,level]-1
            if(i_s < 0): continue
            occupied += 1
            sp_en = np.sum(-h*wf[isospin,i_s,i_l,i_n,:]*hpsi[isospin,i_s,i_l,i_n,:])
            energy[isospin].append(sp_en)
            print("Level: {}, N: {}, L: {}, S: {}".format(level+1,i_n+1,i_l,i_s))
            print("Energy: {}".format(energy[isospin][i]))
        levels[isospin] = occupied
    energies.append(energy)

#h,nbox,lmax,nmax,sortstates,sortenergies,wf = read_wfs()
# npart = 0
# for i in range(nmax):
#     if(sortstates[0,0,i]>0):
#         nfill = int(2*(sortstates[0,1,i]+spin(sortstates[0,2,i])) + 1)
#         if (sortstates[0,1,i] == 0): nfill = 2
#         npart += nfill
#         print(npart)

Level: 1, N: 1, L: 0, S: 0
Energy: -49.71129278586775
Level: 2, N: 1, L: 1, S: 1
Energy: -35.89095593307716
Level: 3, N: 1, L: 1, S: 0
Energy: -33.846486193249426
Level: 4, N: 1, L: 2, S: 1
Energy: -22.12394236411225
Level: 5, N: 2, L: 0, S: 0
Energy: -17.757508449901596
Level: 6, N: 1, L: 2, S: 0
Energy: -17.429233705493058
Level: 7, N: 1, L: 3, S: 1
Energy: -9.231279254081628
Level: 1, N: 1, L: 0, S: 0
Energy: -45.892224179653205
Level: 2, N: 1, L: 1, S: 1
Energy: -34.25186383211593
Level: 3, N: 1, L: 1, S: 0
Energy: -30.43092605315403
Level: 4, N: 1, L: 2, S: 1
Energy: -22.400019151985127
Level: 5, N: 2, L: 0, S: 0
Energy: -16.77249231585236
Level: 6, N: 1, L: 2, S: 0
Energy: -15.312646617349705
Level: 1, N: 1, L: 0, S: 0
Energy: -49.974971340719215
Level: 2, N: 1, L: 1, S: 1
Energy: -36.090274595109754
Level: 3, N: 1, L: 1, S: 0
Energy: -34.02870614028229
Level: 4, N: 1, L: 2, S: 1
Energy: -22.2631911837365
Level: 5, N: 2, L: 0, S: 0
Energy: -17.867634856351053
Level: 6, N: 1, L: 2

Now we can calculate the hpsi (or, G) objects from Wfs. Here I am reading in training Wfs just as a test.

In [9]:
for i in range(len(params)-4):
    if(errors[i] == 1): continue
    # The following is to simply read in the trial WFs during testing
    restartfile = '../HF/results/wf_numpy.bin.'+str(i)
    h,nbox,lmax,nmax,sortstates,sortenergies,wf = read_wfs(restartfile)

    # get hpsi
    hpsi = get_hpsi(params[i],h,nbox,lmax,nmax,sortstates,sortenergies,wf)

    #Calculate energies as a test for levels
    occupied = 0
    for i in range(nmax):
        isospin = 0
        level = i
        i_s = sortstates[isospin,2,level]-1
        i_l = sortstates[isospin,1,level]
        i_n = sortstates[isospin,0,level]-1
        if(i_s < 0): continue
        occupied += 1
        print("Level: {}, N: {}, L: {}, S: {}".format(level+1,i_n+1,i_l,i_s))
        print("Energy: {}".format(np.sum(-h*wf[isospin,i_s,i_l,i_n,:]*hpsi[isospin,i_s,i_l,i_n,:])))

print("There are {} occupied levels!".format(occupied))

Level: 1, N: 1, L: 0, S: 0
Energy: -49.71129278586775
Level: 2, N: 1, L: 1, S: 1
Energy: -35.89095593307716
Level: 3, N: 1, L: 1, S: 0
Energy: -33.846486193249426
Level: 4, N: 1, L: 2, S: 1
Energy: -22.12394236411225
Level: 5, N: 2, L: 0, S: 0
Energy: -17.757508449901596
Level: 6, N: 1, L: 2, S: 0
Energy: -17.429233705493058
Level: 7, N: 1, L: 3, S: 1
Energy: -9.231279254081628
There are 7 occupied levels!


$ \braket{\psi_k\vert G(\phi)} - \lambda \braket{\psi_k\vert\phi} = 0$ 

$ \braket{\phi\vert\phi} - 1 = 0 $

In [46]:
def make_wf(coeffs,train_wfs,states,levels):
    train_num = len(train_wfs)
    tot_levels = sum(levels)
    trial_wf = np.zeros(train_wfs[0].shape)
    for i in range(train_num):
        for isospin in range(2):
            for j in range(levels[isospin]):
                i_s = sortstates[isospin,2,j]-1
                i_l = sortstates[isospin,1,j]
                i_n = sortstates[isospin,0,j]-1
                trial_wf[isospin,i_s,i_l,i_n,:] = trial_wf[isospin,i_s,i_l,i_n,:] \
                    + train_wfs[i][isospin,i_s,i_l,i_n,:] * coeffs[i*tot_levels + isospin*levels[0] + j]
    return trial_wf

def galerkin(coeffs,train_wfs,states,levels,params):
    
    global count
    global prev_time
    
    count += 1
    print("Iteration {}".format(count))
    print("Iteration time: {}".format(time.time() - prev_time))
    
    prev_time = time.time()
    
    results = np.zeros(coeffs.shape)
    
    train_num = len(train_wfs)
    tot_levels = sum(levels)
    
    trial_wf = make_wf(coeffs,train_wfs,states,levels)

    hpsi = get_hpsi(params,h,nbox,lmax,nmax,sortstates,sortenergies,trial_wf)

    for i in range(train_num):
        for isospin in range(2):
            for j in range(levels[isospin]):
                i_s = sortstates[isospin,2,j]-1
                i_l = sortstates[isospin,1,j]
                i_n = sortstates[isospin,0,j]-1
                ele = i*tot_levels + isospin*levels[0] + j
                lam = coeffs[train_num*tot_levels + isospin*levels[0] + j]
                results[ele] = h*np.dot(train_wfs[i][isospin,i_s,i_l,i_n,:],hpsi[isospin,i_s,i_l,i_n,:]) \
                    - lam*h*np.dot(train_wfs[i][isospin,i_s,i_l,i_n,:],trial_wf[isospin,i_s,i_l,i_n,:])


    for isospin in range(2):
        for j in range(levels[isospin]):
            i_s = sortstates[isospin,2,j]-1
            i_l = sortstates[isospin,1,j]
            i_n = sortstates[isospin,0,j]-1
            ele = train_num*tot_levels + isospin*levels[0] + j
            results[ele] = h*np.dot(trial_wf[isospin,i_s,i_l,i_n,:],trial_wf[isospin,i_s,i_l,i_n,:]) - 1.0
            
    return results



In [74]:
global count
count = 0

start_time = time.time()
global prev_time 
prev_time = time.time()

train_num = len(train_wfs)

print("Training with {} WFs, each one with {} total states".format(train_num,sum(levels)))

ak = np.zeros((train_num,sum(levels)))
lam = np.zeros((sum(levels)))
coeffs = np.zeros((train_num*sum(levels) + sum(levels)))
coeffs[0:train_num*sum(levels)] = 0.1
coeffs[train_num*sum(levels):-1] = 25.0
#coeffs = sol.x

test_params = [-2493.45,]
#galerkin(coeffs,wfs,states,levels,test_params)

hybr_dict = {'col_deriv': 0, 'xtol': 1.49012e-8, 'maxfev': 500, 'band': None, 'eps': None, 'factor': 100, 'diag': None}
sol = optimize.root(galerkin, coeffs, args=(train_wfs,states,levels,test_params), method='hybr', options=hybr_dict)
print(sol.x)
print(galerkin(sol.x,train_wfs,states,levels,test_params))
print("Total time took: {}".format(time.time()-start_time))

Training with 4 WFs, each one with 13 total states
Iteration 1
Iteration time: 0.0003044605255126953
Iteration 2
Iteration time: 0.041045188903808594
Iteration 3
Iteration time: 0.039368391036987305
Iteration 4
Iteration time: 0.03859591484069824
Iteration 5
Iteration time: 0.039981842041015625
Iteration 6
Iteration time: 0.04120445251464844
Iteration 7
Iteration time: 0.04121804237365723
Iteration 8
Iteration time: 0.039839982986450195
Iteration 9
Iteration time: 0.04020977020263672
Iteration 10
Iteration time: 0.039960384368896484
Iteration 11
Iteration time: 0.03962421417236328
Iteration 12
Iteration time: 0.04021024703979492
Iteration 13
Iteration time: 0.04119873046875
Iteration 14
Iteration time: 0.03965044021606445
Iteration 15
Iteration time: 0.0394287109375
Iteration 16
Iteration time: 0.039903879165649414
Iteration 17
Iteration time: 0.03854966163635254
Iteration 18
Iteration time: 0.03965568542480469
Iteration 19
Iteration time: 0.03998684883117676
Iteration 20
Iteration tim

Iteration 169
Iteration time: 0.040599822998046875
Iteration 170
Iteration time: 0.04077553749084473
Iteration 171
Iteration time: 0.04095745086669922
Iteration 172
Iteration time: 0.03993034362792969
Iteration 173
Iteration time: 0.0391387939453125
Iteration 174
Iteration time: 0.03743720054626465
Iteration 175
Iteration time: 0.03905677795410156
Iteration 176
Iteration time: 0.04038882255554199
Iteration 177
Iteration time: 0.04039335250854492
Iteration 178
Iteration time: 0.03958010673522949
Iteration 179
Iteration time: 0.04043292999267578
Iteration 180
Iteration time: 0.040497541427612305
Iteration 181
Iteration time: 0.039629459381103516
Iteration 182
Iteration time: 0.04009079933166504
Iteration 183
Iteration time: 0.038399696350097656
Iteration 184
Iteration time: 0.037445068359375
Iteration 185
Iteration time: 0.039469242095947266
Iteration 186
Iteration time: 0.039933204650878906
Iteration 187
Iteration time: 0.039406538009643555
Iteration 188
Iteration time: 0.03773117065429

Iteration 336
Iteration time: 0.04142332077026367
Iteration 337
Iteration time: 0.03970980644226074
Iteration 338
Iteration time: 0.0386962890625
Iteration 339
Iteration time: 0.0392155647277832
Iteration 340
Iteration time: 0.038306236267089844
Iteration 341
Iteration time: 0.041345834732055664
Iteration 342
Iteration time: 0.039997100830078125
Iteration 343
Iteration time: 0.03975057601928711
Iteration 344
Iteration time: 0.03985404968261719
Iteration 345
Iteration time: 0.040050506591796875
Iteration 346
Iteration time: 0.04007554054260254
Iteration 347
Iteration time: 0.03897428512573242
Iteration 348
Iteration time: 0.04008364677429199
Iteration 349
Iteration time: 0.03994941711425781
Iteration 350
Iteration time: 0.0397641658782959
Iteration 351
Iteration time: 0.03912782669067383
Iteration 352
Iteration time: 0.041314125061035156
Iteration 353
Iteration time: 0.03917837142944336
Iteration 354
Iteration time: 0.03957366943359375
Iteration 355
Iteration time: 0.03868603706359863
I

Iteration 499
Iteration time: 0.04059863090515137
Iteration 500
Iteration time: 0.038967132568359375
Iteration 501
Iteration time: 0.038689374923706055
Iteration 502
Iteration time: 0.04026317596435547
Iteration 503
Iteration time: 0.04125046730041504
Iteration 504
Iteration time: 0.03779149055480957
Iteration 505
Iteration time: 0.039293766021728516
Iteration 506
Iteration time: 0.040101051330566406
Iteration 507
Iteration time: 0.039878129959106445
Iteration 508
Iteration time: 0.03901410102844238
Iteration 509
Iteration time: 0.038680315017700195
Iteration 510
Iteration time: 0.04012274742126465
Iteration 511
Iteration time: 0.039544105529785156
[-1.06839913e+01 -6.54793118e+01 -6.00387732e+01  1.25713964e+01
 -2.61406013e+01  3.18043019e+01  2.28405361e+00  1.08612814e+02
  5.32588267e+01 -1.29019549e+02 -1.65884173e+01  3.69479166e+01
  1.33577713e+01  1.12772573e+01  6.38777400e+01  5.86882696e+01
 -1.09038861e+01  2.61059371e+01 -2.93259738e+01 -1.03929382e+00
 -1.03070677e+02 -

In [16]:
# Make the test WF with test_params
make_input(test_params,0)
with cwd('../HF/'):
    errors.append(run_HF("test",0))


In [75]:
# Make trial WF and hpsi with solved coeffs
trial_wf = make_wf(sol.x,train_wfs,states,levels)
trial_hpsi = get_hpsi(test_params,h,nbox,lmax,nmax,sorts,sorte,trial_wf)

# Read in the cheated test
restartfile = '../HF/results/wf_numpy.bin.test'
h,nbox,lmax,nmax,sorts,sorte,test_wf = read_wfs(restartfile)
test_hpsi = get_hpsi(test_params,h,nbox,lmax,nmax,sorts,sorte,test_wf)

for isospin in range(2):
        for j in range(levels[isospin]):
            i_s = sortstates[isospin,2,j]-1
            i_l = sortstates[isospin,1,j]
            i_n = sortstates[isospin,0,j]-1
            norm = np.dot(h*test_wf[isospin,i_s,i_l,i_n,:],trial_wf[isospin,i_s,i_l,i_n,:])
            trial_sp = np.dot(-h*trial_hpsi[isospin,i_s,i_l,i_n,:],trial_wf[isospin,i_s,i_l,i_n,:])
            test_sp = np.dot(-h*test_hpsi[isospin,i_s,i_l,i_n,:],test_wf[isospin,i_s,i_l,i_n,:])
            print("Trial: {trial} Test: {test} Difference: {diff} Norm: {norm}".format(trial=trial_sp,test=test_sp,diff=(trial_sp-test_sp)/trial_sp,norm=norm))



Trial: -51.04364964394536 Test: -51.0389288800412 Difference: 9.248484262175345e-05 Norm: 1.0000040017843066
Trial: -36.89991181264468 Test: -36.89479164177525 Difference: 0.0001387583497605438 Norm: 1.0000029013594778
Trial: -34.76205642491351 Test: -34.76382173761564 Difference: -5.078274658308587e-05 Norm: 1.0000048549171647
Trial: -22.826557025382165 Test: -22.82579417967998 Difference: 3.3419218734367844e-05 Norm: 1.000004835970001
Trial: -18.304214035154963 Test: -18.312348108888745 Difference: -0.00044438257322382594 Norm: 1.000002307000376
Trial: -17.933266595285833 Test: -17.936585636892588 Difference: -0.0001850773582782328 Norm: 1.000004497091181
Trial: -9.667452415084242 Test: -9.668281512421906 Difference: -8.576171902029881e-05 Norm: 1.0000058908460687
Trial: -47.28688170061588 Test: -47.28474752597255 Difference: 4.513248847413362e-05 Norm: 0.9999988459622862
Trial: -35.38063165648504 Test: -35.37811549754066 Difference: 7.111684632459011e-05 Norm: 0.9999974016372367
Tri